Install stable-baselines3 for training algorithms

In [ ]:
!pip install stable_baselines3

Set-up code

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Unzip custom-hopper environment and utils

In [ ]:
!unzip custom_hopper.zip

Import libraries

In [ ]:
import gym
from env.custom_hopper import *
from stable_baselines3 import PPO

In [ ]:
env = gym.make('CustomHopper-source-v0')
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)


Save model for evaluation

In [ ]:
model.save("ppo_standard")